In [181]:
import numpy as np
import time

In [2]:
import sys
sys.path.insert(0, '/home/adrien/MVA/KERNEL/DataChallenge-KernelMethods/Code/Alex/')

from utils import *
from svm import *

# --Data--

In [3]:
folder = '/home/adrien/MVA/KERNEL/DataChallenge-KernelMethods/Data/'

In [4]:
dataset = 0

In [5]:
data = read_data(folder, dataset=dataset)

In [6]:
print(data.keys())

dict_keys(['Xtr_mat50', 'Ytr', 'Xte_mat50', 'Xte', 'Xtr'])


# --Substring kernel--

In [171]:
def K(s, t, gamma, N):
    
    if N < 2:
        print('works only if N > 1')
        return False
    
    S = len(s)
    T = len(t)
    grid = np.ones((S, T, N))
    grid_K = np.zeros((S, T))
    
    for p in range(1, N):
        for i in range(S):
            for j in range(T):
                      
                if min(i+1, j+1) < p:
                    grid[i, j, p] = 0

                elif p == 1 and i == 0:

                    sum_gamma = 0

                    for n, a in enumerate(t[0:j+1]):
                        if a == s[0]:
                            sum_gamma += gamma**(j+1-n)
                    
                    grid[i, j, p] = gamma * sum_gamma

                else:

                    sum_gamma = 0

                    for n, a in enumerate(t[0:j+1]):
                        if a == s[i]:
                            sum_gamma += grid[i-1, n-1, p-1] * (gamma**(j-n+2))

                    grid[i, j, p] = gamma * grid[i-1, j, p] + sum_gamma  
                
    for i in range(S):
        for j in range(T):         
                
            if min(i+1, j+1) < N:
                continue

            else:

                sum_gamma = 0

                for n, a in enumerate(t[0:j+1]):
                    if a == s[i]:
                        sum_gamma += grid[i-1, n-1, N-1] * (gamma**2)

                grid_K[i, j] = grid_K[i-1, j] + sum_gamma

    
    return grid_K[S-1, T-1]  

In [153]:
def alignment(K_1, K_2):
    
    if K_1.shape != K_2.shape:
        return False
    
    return np.sum(K_1*K_2) / np.sqrt(np.sum(K_1*K_1) * np.sum(K_2*K_2))

In [163]:
def normalize(K):
    
    n = K.shape[0]
    k = np.expand_dims(1./np.sqrt(np.diag(K)), 1)
    kkt = np.dot(k, k.T)
    K = K * kkt
    return K

# --Approx kernel--

In [183]:
nb_points = 2000

In [184]:
k = 4
l = 0.8

In [185]:
Xtr = data['Xtr'][0:nb_points]

In [186]:
alphabet = ['A', 'C', 'T', 'G']

In [187]:
substrings = []

In [188]:
for i1 in alphabet:
    for i2 in alphabet:
        for i3 in alphabet:
            for i4 in alphabet:
                
                if i1+i2+i3+i4 not in substrings:
                    substrings.append(i1+i2+i3+i4)

In [189]:
def gram_matrix(X, substrings, l, k):
    """
    X: input data matrix
    substrings: list of possible substrings
    l, k: parameter of kernel
    """
    
    N = len(X)
    M = len(substrings)
    features = np.zeros((N, M))
    
    start = time.time()
    
    for i in tqdm(range(N)):
        for j in range(M):
            
            features[i, j] = K(X[i], substrings[j], l, k)
    
    K_matrix = np.zeros((N, N))
    
    for i in range(N):
    
        K_matrix[i, i] = features[i, :].T.dot(features[i, :]) 
        for j in range(i+1, N):

            K_matrix[i, j] = features[i, :].T.dot(features[j, :]) 
            K_matrix[j, i] = K_matrix[i, j]
    
    print('Gram matrix computed in {} min'.format((time.time() - start)/float(60)))
    
    return K_matrix

In [ ]:
K_approx = gram_matrix(Xtr, substrings, l, k)

 15%|█▌        | 307/2000 [02:43<15:02,  1.88it/s]

In [190]:
np.savetxt('K0_4_08.txt', normalize(K_approx))

# --Main--

parameters

In [6]:
test_size = 0.25
tau = 1e-6
k = 4
gamma = 0.5

data

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = data['Xtr']
Y = data['Ytr']

In [9]:
X_train, X_test, Y_train, Y_test = (train_test_split(X, Y, test_size=test_size))

model - train

In [11]:
from dev import *

In [ ]:
Kernel = Kernel_Substring(k, gamma)
svm_dual = kernel_SVM(tau, X_train, Y_train, Kernel, dual=True)

model - test

In [ ]:
x_sol, alpha, acc_train = svm_dual.svm_solver(solver='cvxopt')
acc_test = svm_dual.compute_accuracy(X_test, Y_test, alpha)
print('acc_train', acc_train)
print('acc_test', acc_test)
print('DUAL OPTIMIZATION FINISHED')

In [16]:
len(X_train[0])

100